## xargs

In [ ]:
%%bash
find test/  -name "*.*" | xargs sed -i 's/\r//g

## tr

In [ ]:
%%bash
echo LuVx21 | tr '[:upper:]' '[:lower:]'

## uname

In [ ]:
%%bash
uname -a

echo "--------------------------------------------------------------------------------------------------------------------------------"
# 系统类型相关
uname -s
uname -o
uname -r
uname -v

echo "--------------------------------------------------------------------------------------------------------------------------------"
# 架构
uname -m # 或arch
# 主机名
uname -n
uname -p
uname -i

## getopts

`getopts optstring [arg]`

optstring是一个字符串, 包含一个可以为getopts命令识别的选项名称列表.

用s表示一个字符, 其中语法为:

| 选项内容 | 说明                                                  |
| :------- | :---------------------------------------------------- |
| :        | optsring如果以:开头, 表示是静默模式, 忽略一般错误消息 |
| s        | 有效选项并且后面不带参数值                            |
| s:       | 有效选项并且后面必须带参数值                          |

In [ ]:
%%bash
function m0() {
    echo "初始选项位置 $OPTIND"
    while getopts ":ab:x::" arg; do
        echo "选项位置 $OPTIND"
        case "$arg" in
            "a")
                echo "选项 $arg 是个标识"
                ;;
            "b")
                echo "选项 $arg 值: $OPTARG"
                ;;
            "?")
                echo "未知选项 $OPTARG"
                ;;
            ":")
                echo "No argument value for option $OPTARG"
                ;;
            *)                 #出现上述枚举项之外的项, 会进入该分支
                echo "Unknown error while processing options"
                ;;
        esac
    done
}
m0 -a -b bar

## getopt

`getopt [options] optstring parameters`

* options: getopt自带的参数定义, 用来表示长选项还是短选项

`-o`或`–options`选项后面是可接受的短选项

`-l`或`–long`选项后面是可接受的长选项

* optstring: 自定义选项参数, 例如 -a -b

选项后面可以跟 一个冒号(:)、两个冒号、没冒号

* parameters: 参数, 即参数值, 允许为空


选项后一个冒号(:)、两个冒号、没冒号的区别

* 无冒号: 执行时, 只有选项, 不带参数.
* 一个冒号(:): 执行时必须带有参数(必选).
* 两个冒号(::): 执行时可以选择性带参数, 也可以不带(可选).


getopt支持短选项和长选项:

* 短选项格式为 -a -b, 即长度为1的字母
* 长选项格式为 -name, 即多个字母
* 如果参数是必选, 那么短选项的参数值可以是空格, 也可以紧贴选项, `-c arg`或`-carg`;  长选项的参数可以是空格, 也可以=连接, `–clong arg` 或`clong=arg`
* 如果参数是可选, 那么短选项的参数值只可以紧贴选项, `-carg`; 长选项的参数值只可以=连接, `-clong=arg`

In [ ]:
%%bash
function m1() {
    echo "原始参数: [$@]"

    #-o或--options选项后面是可接受的短选项, 如ab:c::, 表示可接受的短选项为-a -b -c,
    #其中-a选项不接参数, -b选项后必须接参数, -c选项的参数为可选的
    #-l或--long选项后面是可接受的长选项, 用逗号分开, 冒号的意义同短选项.
    #-n选项后接选项解析错误时提示的脚本名字
    ARGS=`getopt -o "hab:c::" --long "help,along,blong:,clong::" -n "$0" -- "$@"`
    if [ $? != 0 ]; then
        echo "Terminating..."
        exit 1
    fi

    #将规范化后的命令行参数分配至位置参数($1,$2,...)
    eval set -- "${ARGS}"
    echo "[$ARGS] 格式化-> [$@]"

    while true; do
    echo "------选项: $1, 值: $2------"
        case "$1" in
            -a|--along)
                echo "选项 a, 无值";
                shift
                ;;
            -b|--blong)
                echo "选项 b, 值: $2";
                shift 2
                ;;
            -c|--clong)
                echo "第二个 $2"
                case "$2" in
                    "")
                        echo "选项 c, 无值";
                        shift 2
                        ;;
                    *)
                        echo "选项 c, 值: $2";
                        shift 2;
                        ;;
                esac
                ;;
            -h|--help)
                echo "帮助信息"
                exit 0
                ;;
            --)
                shift
                break
                ;;
            *)
                echo "Internal error!"
                exit 1
                ;;
        esac
    done
}
# m1 --help
# m1 -h
m1 -a foo -b bar -cfoobar
echo -e "\n"
m1 -a foo -b bar --clong=foobar

# 扩展

## jq

In [ ]:
%%bash
# apt-get -y update && apt-get -y install jq
# curl https://api.github.com/repos/jqlang/jq/releases/latest | jq '.assets[] | if .name == "jq-linux-amd64" then .browser_download_url else "" end'

# cat > /tmp/1.json <<- EOF
# {
#    "manifests": [
#       {
#          "digest": "sha256:9dfca77839884865eefc3d2ebce38ea598f59e8cbb843e345bbd7b135fb54759",
#          "platform": {
#             "architecture": "amd64",
#             "os": "linux"
#          }
#       },
#       {
#          "digest": "sha256:484b6dd7aea82ca0597cfe83e0e5309657c6612a6fb35957762eb62f2ab0818c",
#          "platform": {
#             "architecture": "arm64",
#             "os": "linux"
#          }
#       },
#       {
#          "digest": "sha256:8eea69d1a6e473e9876b0f73dcf9141dcf4a3f1d8b0c11815c813b827e820839",
#          "platform": {
#             "architecture": "unknown",
#             "os": "unknown"
#          }
#       }
#    ]
# }
# EOF

In [ ]:
%%bash
# cat /tmp/1.json | jq '.manifests[] | if .platform.os == "linux" and (.platform.architecture == "amd64" or .platform.architecture == "arm64") then .digest else "" end'
cat /tmp/1.json | jq '.manifests[] | select(.platform.os == "linux") | select(.platform.architecture == "amd64" or .platform.architecture == "arm64") | .digest'